# 诗歌生成

# 数据处理

In [10]:
import tensorflow as tf
tf.enable_eager_execution() # 关键
import numpy as np
import collections
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import layers, optimizers, datasets
start_token = 'bos'
end_token = '夜'

def process_dataset(fileName):
    examples = []
    with open(fileName, 'r') as fd:
        for line in fd:
            outs = line.strip().split(':')
            content = ''.join(outs[1:])
            ins = [start_token] + list(content) + [end_token] 
            if len(ins) > 200:
                continue
            examples.append(ins)
            
    counter = collections.Counter()
    for e in examples:
        for w in e:
            counter[w]+=1
    
    sorted_counter = sorted(counter.items(), key=lambda x: -x[1])  # 排序
    words, _ = zip(*sorted_counter)
    words = ('PAD', 'UNK') + words[:len(words)]
    word2id = dict(zip(words, range(len(words))))
    id2word = {word2id[k]:k for k in word2id}
    
    indexed_examples = [[word2id[w] for w in poem]
                        for poem in examples]
    seqlen = [len(e) for e in indexed_examples]
    tf.config.experimental.list_physical_devices('GPU')
    instances = list(zip(indexed_examples, seqlen))
    
    return instances, word2id, id2word

def poem_dataset():
    instances, word2id, id2word = process_dataset('./poems.txt')
    ds = tf.data.Dataset.from_generator(lambda: [ins for ins in instances], 
                                            (tf.int64, tf.int64), 
                                            (tf.TensorShape([None]),tf.TensorShape([])))
    ds = ds.shuffle(buffer_size=10240)
    ds = ds.padded_batch(100, padded_shapes=(tf.TensorShape([None]),tf.TensorShape([])))
    ds = ds.map(lambda x, seqlen: (x[:, :-1], x[:, 1:], seqlen-1))
    return ds, word2id, id2word

# 模型代码， 完成建模代码

In [11]:
class myRNNModel(keras.Model):
    def __init__(self, w2id):
        super(myRNNModel, self).__init__()
        self.v_sz = len(w2id)
        self.embed_layer = tf.keras.layers.Embedding(self.v_sz, 64, 
                                                    batch_input_shape=[None, None])
        
        self.rnncell = tf.keras.layers.SimpleRNNCell(128)
        self.rnn_layer = tf.keras.layers.RNN(self.rnncell, return_sequences=True)
        self.dense = tf.keras.layers.Dense(self.v_sz)
        
    @tf.function
    def call(self, inp_ids, training=None):
        '''
        此处完成建模过程
        '''
        inp_emb = self.embed_layer(inp_ids) # shape(b_sz, seq_len, emb_sz)
        rnn_output = self.rnn_layer(inp_emb, training=training) # shape(b_sz, seq_len, h_sz)
        logits = self.dense(rnn_output) # shape(b_sz, seq_len, v_sz)
        return logits
    
    @tf.function
    def get_next_token(self, x, state=None):
        '''
        生成序列中的下一个token
        '''
        inp_emb = self.embed_layer(x) # shape(b_sz, emb_sz)
        if state is None:
            state = self.rnncell.get_initial_state(inp_emb)
        h, state = self.rnncell(inp_emb, state) # shape(b_sz, h_sz)
        logits = self.dense(h) # shape(b_sz, v_sz)
        out = tf.argmax(logits, axis=-1)
        return out, state


## 一个计算sequence loss的辅助函数，只需了解用途。

In [12]:
def mkMask(input_tensor, maxLen):
    shape_of_input = tf.shape(input_tensor)
    shape_of_output = tf.concat(axis=0, values=[shape_of_input, [maxLen]])

    oneDtensor = tf.reshape(input_tensor, shape=(-1,))
    flat_mask = tf.sequence_mask(oneDtensor, maxlen=maxLen)
    return tf.reshape(flat_mask, shape_of_output)


def reduce_avg(reduce_target, lengths, dim):
    """
    Args:
        reduce_target : shape(d_0, d_1,..,d_dim, .., d_k)
        lengths : shape(d0, .., d_(dim-1))
        dim : which dimension to average, should be a python number
    """
    shape_of_lengths = lengths.get_shape()
    shape_of_target = reduce_target.get_shape()
    if len(shape_of_lengths) != dim:
        raise ValueError(('Second input tensor should be rank %d, ' +
                         'while it got rank %d') % (dim, len(shape_of_lengths)))
    if len(shape_of_target) < dim+1 :
        raise ValueError(('First input tensor should be at least rank %d, ' +
                         'while it got rank %d') % (dim+1, len(shape_of_target)))

    rank_diff = len(shape_of_target) - len(shape_of_lengths) - 1
    mxlen = tf.shape(reduce_target)[dim]
    mask = mkMask(lengths, mxlen)
    if rank_diff!=0:
        len_shape = tf.concat(axis=0, values=[tf.shape(lengths), [1]*rank_diff])
        mask_shape = tf.concat(axis=0, values=[tf.shape(mask), [1]*rank_diff])
    else:
        len_shape = tf.shape(lengths)
        mask_shape = tf.shape(mask)
    lengths_reshape = tf.reshape(lengths, shape=len_shape)
    mask = tf.reshape(mask, shape=mask_shape)

    mask_target = reduce_target * tf.cast(mask, dtype=reduce_target.dtype)

    red_sum = tf.reduce_sum(mask_target, axis=[dim], keepdims=False)
    red_avg = red_sum / (tf.cast(lengths_reshape, dtype=tf.float32) + 1e-30)
    return red_avg

# 定义loss函数，定义训练函数

In [13]:
@tf.function
def compute_loss(logits, labels, seqlen):
    losses = tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels)
    losses = reduce_avg(losses, seqlen, dim=1)
    return tf.reduce_mean(losses)

@tf.function
def train_one_step(model, optimizer, x, y, seqlen):
    '''
    完成一步优化过程，可以参考之前做过的模型
    '''
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y, seqlen)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

def train(epoch, model, optimizer, ds):
    loss = 0.0
    accuracy = 0.0
    for step, (x, y, seqlen) in enumerate(ds):
        loss = train_one_step(model, optimizer, x, y, seqlen)

        if step % 500 == 0:
            print('epoch', epoch, ': loss', loss.numpy())

    return loss


# 训练优化过程

In [14]:
optimizer = optimizers.Adam(0.0005)
train_ds, word2id, id2word = poem_dataset()
model = myRNNModel(word2id)


for epoch in range(10):
    loss = train(epoch, model, optimizer, train_ds)

2024-03-25 16:13:25.937597: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: __inference_Dataset_from_generator_get_iterator_id_fn_48891
2024-03-25 16:13:25.937643: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: __inference_Dataset_from_generator_generator_next_fn_48898
2024-03-25 16:13:25.937659: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: __inference_Dataset_from_generator_finalize_fn_48903


2024-03-25 16:13:26.275967: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: rnn_1_while_cond_49074
2024-03-25 16:13:26.276014: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: rnn_1_while_body_49075
2024-03-25 16:13:26.328496: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: rnn_1_while_cond_49265
2024-03-25 16:13:26.328540: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: rnn_1_while_body_49266


2024-03-25 16:13:26.487305: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: __inference_call_49363
2024-03-25 16:13:26.487516: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: rnn_1_while_cond_49265
2024-03-25 16:13:26.487535: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: rnn_1_while_body_49266
2024-03-25 16:13:26.487588: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: __inference_compute_loss_49430
2024-03-25 16:13:26.487691: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: __forward_compute_loss_49436
2024-03-25 16:13:26.487792: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: __inference___backward_compute_loss_49434_49491
2024-03-25 16:13:26.487882: W tensorflow/core/common_runtime/eager/context.cc:371] Ad

epoch 0 : loss 8.820492


2024-03-25 16:13:26.911954: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: rnn_1_while_cond_50523
2024-03-25 16:13:26.911996: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: rnn_1_while_body_50524
2024-03-25 16:13:27.316424: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: __inference_call_50621
2024-03-25 16:13:27.316591: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: rnn_1_while_cond_50523
2024-03-25 16:13:27.316607: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: rnn_1_while_body_50524
2024-03-25 16:13:27.316644: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: __inference_compute_loss_50687
2024-03-25 16:13:27.316706: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same

...

2024-03-25 16:14:42.006176: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: rnn_1_while_cond_82757
2024-03-25 16:14:42.006215: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: rnn_1_while_body_82758
2024-03-25 16:14:42.138908: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: __inference_call_82855
2024-03-25 16:14:42.139197: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: rnn_1_while_cond_82757
2024-03-25 16:14:42.139216: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: rnn_1_while_body_82758
2024-03-25 16:14:42.139298: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: __inference_compute_loss_82921
2024-03-25 16:14:42.139429: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same

2024-03-25 16:14:44.291303: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: rnn_1_while_cond_83622
2024-03-25 16:14:44.291341: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: rnn_1_while_body_83623
2024-03-25 16:14:44.420356: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: __inference_call_83720
2024-03-25 16:14:44.420578: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: rnn_1_while_cond_83622
2024-03-25 16:14:44.420599: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: rnn_1_while_body_83623
2024-03-25 16:14:44.420662: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: __inference_compute_loss_83786
2024-03-25 16:14:44.420716: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same

2024-03-25 16:14:46.935998: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: rnn_1_while_cond_84491
2024-03-25 16:14:46.936043: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: rnn_1_while_body_84492
2024-03-25 16:14:47.084940: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: __inference_call_84589
2024-03-25 16:14:47.085215: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: rnn_1_while_cond_84491
2024-03-25 16:14:47.085240: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: rnn_1_while_body_84492
2024-03-25 16:14:47.085318: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: __inference_compute_loss_84655
2024-03-25 16:14:47.085375: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same

2024-03-25 16:14:49.182832: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: rnn_1_while_cond_85344
2024-03-25 16:14:49.182878: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: rnn_1_while_body_85345
2024-03-25 16:14:49.321778: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: __inference_call_85442
2024-03-25 16:14:49.321919: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: rnn_1_while_cond_85344
2024-03-25 16:14:49.321942: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: rnn_1_while_body_85345
2024-03-25 16:14:49.322025: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same name: __inference_compute_loss_85508
2024-03-25 16:14:49.322093: W tensorflow/core/common_runtime/eager/context.cc:371] Added two functions with the same

epoch 1 : loss 6.565371
epoch 2 : loss 6.0079875
epoch 3 : loss 5.8450575
epoch 4 : loss 5.698929
epoch 5 : loss 5.573985
epoch 6 : loss 5.489521
epoch 7 : loss 5.4109817
epoch 8 : loss 5.340436
epoch 9 : loss 5.2828493


# 生成过程

In [20]:
def gen_sentence():
    state = [tf.random.normal(shape=(1, 128), stddev=0.5), tf.random.normal(shape=(1, 128), stddev=0.5)]
    cur_token = tf.constant([word2id['bos']], dtype=tf.int32)
    collect = []
    for _ in range(50):
        cur_token, state = model.get_next_token(cur_token, state)
        collect.append(cur_token.numpy()[0])
    return [id2word[t] for t in collect]
print(''.join(gen_sentence()))

一片云中，不得人间。夜日无人，不得人间。夜日无人不可怜，一枝风雨满花声。夜来不得人间处，不得人间不得
